# EDA

In this Jupyter notebook, we embark on a journey to analyze sentiment in tweets related to Apple and Google products. Our goals are to understand the dataset's structure, clean and preprocess the data, and prepare it for a machine learning model that predicts sentiment based on tweet content. Let's start by loading the dataset and performing an exploratory data analysis (EDA).


In [1]:
#Import the necessary packages
import pandas as pd

## Load and Explore the Dataset
First, we load the dataset to understand its basic structure, including the number of entries, columns, and types of data it contains. This initial exploration is crucial for planning our preprocessing steps.

The dataset contains three columns:
- `tweet_text`: The text of the tweet.
- `emotion_in_tweet_is_directed_at`: The product or brand the tweet is directed at (e.g., iPhone, iPad, Google).
- `is_there_an_emotion_directed_at_a_brand_or_product`: The sentiment of the tweet (e.g., Positive emotion, Negative emotion).



In [2]:
data_path = 'data/judge-1377884607_tweet_product_company.csv'
tweets_df = pd.read_csv(data_path, encoding='ISO-8859-1')
print("Dataset shape:", tweets_df.shape)
print("Data types:\n", tweets_df.dtypes)
tweets_df.head()

Dataset shape: (9093, 3)
Data types:
 tweet_text                                            object
emotion_in_tweet_is_directed_at                       object
is_there_an_emotion_directed_at_a_brand_or_product    object
dtype: object


,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


## Missing Values Analysis
Identifying and handling missing values is crucial since they can significantly impact the performance of our model.

The dataset contains missing values, primarily in the `emotion_in_tweet_is_directed_at` column, with a smaller number in the `tweet_text` column. Since our primary focus is sentiment analysis based on the tweet text, we'll proceed by dropping rows where the tweet text is missing. The `emotion_in_tweet_is_directed_at column` can be ignored for our current purpose, as we are focusing on sentiment, not the specific product mentioned.


In [3]:
missing_values = tweets_df.isnull().sum()
print("Missing values in each column:\n", missing_values)


Missing values in each column:
 tweet_text                                               1
emotion_in_tweet_is_directed_at                       5802
is_there_an_emotion_directed_at_a_brand_or_product       0
dtype: int64


In [4]:
# Remove rows with null values in the 'tweet_text' column in the original DataFrame
tweets_df = tweets_df.dropna(subset=['tweet_text'])

## Sentiment Distribution
Understanding the balance between different sentiment classes helps us gauge the dataset's bias towards certain sentiments. This is important for selecting appropriate modeling and resampling techniques. 

The `is_there_an_emotion_directed_at_a_brand_or_product` column has four unique labels:

- Negative emotion
- Positive emotion
- No emotion toward brand or product
- I can't tell

For the sentiment analysis model, we'll focus on positive and negative emotions. We'll treat "No emotion toward brand or product" and "I can't tell" as neutral or unknown sentiments, which might be excluded from the training to focus the model on distinguishing clearly between positive and negative sentiments.

We have 2,978 positive and 570 negative sentiment tweets. This imbalance in the dataset could influence the performance of our model, making it potentially biased towards predicting positive sentiments. We'll need to keep this in mind during the modeling phase, possibly using techniques like oversampling the minority class, undersampling the majority class, or adjusting the class weights in the model training process to address this imbalance.

In [5]:
sentiment_distribution = tweets_df['is_there_an_emotion_directed_at_a_brand_or_product'].value_counts()
print("Sentiment distribution:\n", sentiment_distribution)


Sentiment distribution:
 No emotion toward brand or product    5388
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: int64


## Text Length Analysis
Analyzing the length of tweets can reveal insights about the dataset's variability. It helps us decide if we need to normalize text lengths through padding or truncation during preprocessing.


In [6]:
tweets_df['text_length'] = tweets_df['tweet_text'].apply(lambda x: len(str(x)))
tweets_df['text_length'].describe()


count    9092.000000
mean      104.962275
std        27.187640
min        11.000000
25%        86.000000
50%       109.000000
75%       126.000000
max       178.000000
Name: text_length, dtype: float64

## Frequent Words Analysis
Identifying the most frequently occurring words will help us understand common themes and refine our list of stopwords. High-frequency words with little sentiment value might be removed to improve model accuracy.
|

In [7]:
from collections import Counter
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

all_words = ' '.join(tweets_df['tweet_text'].dropna()).lower()
all_words_tokenized = word_tokenize(all_words)

word_counts = Counter(all_words_tokenized)
word_counts.most_common(20)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\johns\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[('#', 15875),
 ('sxsw', 9516),
 ('@', 7194),
 ('mention', 7124),
 ('.', 5506),
 ('the', 4424),
 ('link', 4313),
 ('}', 4298),
 ('{', 4296),
 ('to', 3586),
 (',', 3533),
 ('at', 3102),
 ('rt', 2962),
 (';', 2800),
 ('&', 2707),
 ('google', 2595),
 ('for', 2545),
 ('ipad', 2446),
 ('!', 2398),
 ('a', 2312)]

## Special Characters and URLs Analysis
Tweets often contain special characters, emojis, and URLs that may not contribute to sentiment analysis. Deciding whether to keep or remove these can impact preprocessing steps and ultimately model performance.


In [8]:
import re

special_chars = tweets_df['tweet_text'].apply(lambda x: re.findall(r'[^\w\s]', str(x)))
special_chars_counts = Counter([item for sublist in special_chars for item in sublist])
print("Special characters counts:", special_chars_counts.most_common(10))

urls_counts = tweets_df['tweet_text'].apply(lambda x: len(re.findall(r"http\S+|www\S+|https\S+", str(x)))).sum()
print("Number of URLs:", urls_counts)


Special characters counts: [('#', 15875), ('.', 8382), ('@', 7194), ('}', 4298), ('{', 4296), (',', 3558), ("'", 2903), (';', 2800), ('&', 2707), ('-', 2438)]
Number of URLs: 44


# Preprocessing


## Encoding Sentiment Labels
For our model to understand the sentiment labels, we need to encode them into a numerical format. In our case, we'll encode positive sentiment as 1 and negative sentiment as 0. This step is crucial for the subsequent modeling process.


In [9]:
sentiment_mapping = {
    'Positive emotion': 1,
    'Negative emotion': 0
}

# Update the mapping based on your dataset's sentiment labels
tweets_df['sentiment_label'] = tweets_df['is_there_an_emotion_directed_at_a_brand_or_product'].map(sentiment_mapping)


In [10]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9092 entries, 0 to 9092
Data columns (total 5 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   tweet_text                                          9092 non-null   object 
 1   emotion_in_tweet_is_directed_at                     3291 non-null   object 
 2   is_there_an_emotion_directed_at_a_brand_or_product  9092 non-null   object 
 3   text_length                                         9092 non-null   int64  
 4   sentiment_label                                     3548 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 426.2+ KB


## Handling Missing Values
After cleaning the text, we need to address missing values. Missing sentiment labels or texts could mislead our model training. Depending on the context, we might choose to fill them with a placeholder value or remove those entries.


In [11]:
# Dropping rows with any missing sentiment labels or cleaned tweets
tweets_df = tweets_df.dropna(subset=['sentiment_label'])
tweets_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3548 entries, 0 to 9088
Data columns (total 5 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   tweet_text                                          3548 non-null   object 
 1   emotion_in_tweet_is_directed_at                     3191 non-null   object 
 2   is_there_an_emotion_directed_at_a_brand_or_product  3548 non-null   object 
 3   text_length                                         3548 non-null   int64  
 4   sentiment_label                                     3548 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 166.3+ KB


# Splitting the Data
Before training our model, we'll split the dataset into training and testing sets. This allows us to train our model on one subset of the data and then test its performance on unseen data, providing a better evaluation of its real-world performance.


In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    tweets_df['tweet_text'],
    tweets_df['sentiment_label'],
    stratify=tweets_df['sentiment_label'],
    test_size=0.2, 
    random_state=42
)


## Cleaning the Text Data (BorA TTSplit?)
The next step in our preprocessing is to clean the text data. This involves removing URLs, mentions, special characters, and numbers. These elements are generally not useful for sentiment analysis and can introduce noise into our model. Cleaning helps in focusing on the meaningful content of the tweets.


In [13]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def clean_tweet(tweet):
    # Lowercase the text
    tweet = tweet.lower()
    
    # Remove URLs
    tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet, flags=re.MULTILINE)
    
    # Remove punctuation
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    
    # Remove mentions and hashtags
    tweet = re.sub(r'@\w+|#\w+', '', tweet)
    
    # Tokenization
    tokens = word_tokenize(tweet)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Remove numeric characters
    tokens = [word for word in tokens if not word.isdigit()]
    
    # Join tokens back into a string
    clean_tweet = ' '.join(tokens)
    
    return clean_tweet


X_train = X_train.apply(clean_tweet)
X_test = X_test.apply(clean_tweet)
X_train.head()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\johns\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\johns\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\johns\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


7003    someone buy ipad v1 wait line get v2 tomorrow ...
8981    think effing hubby line ipad someone point tow...
7351    smart company rt mention rumor apple opening t...
7483                   google map mobile look awesomesxsw
1642    mention mention wishing excellent day sxsw tod...
Name: tweet_text, dtype: object

In [14]:
# import re

# def clean_tweet(tweet):
#     tweet = str(tweet).lower() #Lower case everything
#     tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)  # Remove URLs
#     tweet = re.sub(r'\@\w+|\#', '', tweet)  # Remove mentions and keep hashtags
#     tweet = re.sub(r'\d+', '', tweet)  # Remove numbers
#     tweet = re.sub(r'[^\w\s]', '', tweet)  # Remove punctuation
#     return tweet

# # tweets_df['cleaned_tweet_text'] = tweets_df['tweet_text'].apply(clean_tweet)

# X_train = X_train.apply(clean_tweet)
# X_test = X_test.apply(clean_tweet)


# # def preprocess_text(text):
# #     # Tokenize text
# #     tokens = word_tokenize(text)
# #     # Remove punctuation
# #     tokens = [token for token in tokens if token not in string.punctuation]
# #     # Convert tokens to lowercase
# #     tokens = [token.lower() for token in tokens]
# #     # Lemmatize tokens
# #     tokens = [lemmatizer.lemmatize(token) for token in tokens]
# #     # Remove stop words
# #     stop_words = set(stopwords.words('english'))
# #     tokens = [token for token in tokens if token not in stop_words]
# #     # Join tokens back into a single string
# #     preprocessed_text = ' '.join(tokens)
# #     return preprocessed_text


In [15]:
X_train.head()

7003    someone buy ipad v1 wait line get v2 tomorrow ...
8981    think effing hubby line ipad someone point tow...
7351    smart company rt mention rumor apple opening t...
7483                   google map mobile look awesomesxsw
1642    mention mention wishing excellent day sxsw tod...
Name: tweet_text, dtype: object

## Leminization (BorA TTSplit?)

In [16]:
# from nltk.stem import WordNetLemmatizer
# # Lemmatization function
# lemmatizer = WordNetLemmatizer()
# def lemmatize_text(text):
#     return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

# # Apply cleaning and lemmatization
# X_train = X_train.apply(lemmatize_text)
# X_test = X_test.apply(lemmatize_text)

# # Or 

# # tweets_df['cleaned_tweet_text'] = tweets_df['tweet_text'].apply(lemmatize_text)

In [17]:
X_train.head()

7003    someone buy ipad v1 wait line get v2 tomorrow ...
8981    think effing hubby line ipad someone point tow...
7351    smart company rt mention rumor apple opening t...
7483                   google map mobile look awesomesxsw
1642    mention mention wishing excellent day sxsw tod...
Name: tweet_text, dtype: object

## Text Vectorization (TF-IDF) (BorA TTSplit?)
Machine Learning models require numerical input, so we convert our cleaned text data into a numerical format using TF-IDF (Term Frequency-Inverse Document Frequency). This technique reflects how important a word is to a document in a collection, helping us to weigh the terms accordingly.


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords

# tfidf_vectorizer = TfidfVectorizer(stop_words=stopwords.words('english'), max_features=10000)
# X_tfidf = tfidf_vectorizer.fit_transform(tweets_df['cleaned_tweet_text'])

# or
# Vectorization with TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words=stopwords.words('english'), max_features=10000, )
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

## Handling Class Imbalance
Our EDA revealed a class imbalance in the sentiment labels. To address this, we can use oversampling for the minority class to help in preventing our model from being biased towards the more frequent class.


In [19]:
from imblearn.over_sampling import RandomOverSampler

#Initialize the RandomOverSampler
ros = RandomOverSampler(random_state=42)

#Resample the training data
X_train_resampled, y_train_resampled = ros.fit_resample(X_train_tfidf, y_train)

# Check the balance of the resampled data
resampled_balance = pd.Series(y_train_resampled).value_counts()

X_train_resampled.shape, resampled_balance

((4764, 5203),
 0.0    2382
 1.0    2382
 Name: sentiment_label, dtype: int64)

In [20]:
from sklearn.model_selection import cross_val_score
import pandas as pd
from sklearn.pipeline import Pipeline


# Assuming you've already defined your X (features) and y (target) from the cleaned data

# List to keep track of model performance
model_performance = []

# Function to evaluate a model and add its performance to our list
def evaluate_model(model, model_name, X, y, cv=5):
    scores = cross_val_score(model, X, y, cv=cv, scoring='accuracy')
    model_performance.append({
        'Model': model_name,
        'Accuracy Mean': scores.mean(),
        'Accuracy Std': scores.std()
    })

# Example models to evaluate
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Define pipelines for different models
nb_pipeline = Pipeline([('clf', MultinomialNB())])
cnb_pipeline = Pipeline([('clf', ComplementNB())])
lr_pipeline = Pipeline([('clf', LogisticRegression(random_state=42))])
rf_pipeline = Pipeline([('clf', RandomForestClassifier(random_state=42))])

# Evaluate models
evaluate_model(nb_pipeline, 'Multinomial Naive Bayes', X_train_resampled, y_train_resampled)
evaluate_model(cnb_pipeline, 'Complement Naive Bayes', X_train_resampled, y_train_resampled)
evaluate_model(lr_pipeline, 'Logistic Regression', X_train_resampled, y_train_resampled)
evaluate_model(rf_pipeline, 'Random Forest', X_train_resampled, y_train_resampled)

# Convert the performance list to a DataFrame
performance_df = pd.DataFrame(model_performance)

# Display the performance DataFrame
print(performance_df)

                     Model  Accuracy Mean  Accuracy Std
0  Multinomial Naive Bayes       0.915195      0.010337
1   Complement Naive Bayes       0.915195      0.010337
2      Logistic Regression       0.920442      0.008823
3            Random Forest       0.971453      0.001218


In [21]:
from sklearn.model_selection import cross_validate
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline


# Assuming you've already defined your X (features) and y (target) from the cleaned data

# List to keep track of model performance
model_performance = []

# Function to evaluate a model and add its performance to our list
def evaluate_model(model, model_name, X, y, cv=5):
    scores = cross_validate(model, X, y, cv=cv, scoring='accuracy', return_train_score = True)
    model_performance.append({
        'Model': model_name,
        'CV Train Accuracy Mean': np.mean(scores['train_score']),
        'CV Test Accuracy Mean': np.mean(scores['test_score']),
        'CV Test Accuracy Std': np.std(scores['test_score'])
    })

# Example models to evaluate
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Define pipelines for different models
nb_pipeline = Pipeline([('clf', MultinomialNB())])
cnb_pipeline = Pipeline([('clf', ComplementNB())])
lr_pipeline = Pipeline([('clf', LogisticRegression(random_state=42))])
rf_pipeline = Pipeline([('clf', RandomForestClassifier(random_state=42))])


# Evaluate models
evaluate_model(nb_pipeline, 'Multinomial Naive Bayes', X_train_resampled, y_train_resampled)
evaluate_model(cnb_pipeline, 'Complement Naive Bayes', X_train_resampled, y_train_resampled)
evaluate_model(lr_pipeline, 'Logistic Regression', X_train_resampled, y_train_resampled)
evaluate_model(rf_pipeline, 'Random Forest', X_train_resampled, y_train_resampled)

# Convert the performance list to a DataFrame
performance_df = pd.DataFrame(model_performance)

# Display the performance DataFrame
print(performance_df)

                     Model  CV Train Accuracy Mean  CV Test Accuracy Mean  \
0  Multinomial Naive Bayes                0.962479               0.915195   
1   Complement Naive Bayes                0.962741               0.915195   
2      Logistic Regression                0.963948               0.920442   
3            Random Forest                1.000000               0.971453   

   CV Test Accuracy Std  
0              0.010337  
1              0.010337  
2              0.008823  
3              0.001218  


In [22]:
from sklearn.metrics import accuracy_score

# Train the Random Forest pipeline on the entire training dataset
nb_pipeline.fit(X_train_resampled, y_train_resampled)

# Predict on the test set using the Random Forest pipeline
y_pred_test = nb_pipeline.predict(X_test_tfidf)

# Calculate accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred_test)

print("Test Accuracy:", test_accuracy)


Test Accuracy: 0.8661971830985915


In [23]:
from sklearn.metrics import accuracy_score

# Train the Random Forest pipeline on the entire training dataset
cnb_pipeline.fit(X_train_resampled, y_train_resampled)

# Predict on the test set using the Random Forest pipeline
y_pred_test = cnb_pipeline.predict(X_test_tfidf)

# Calculate accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred_test)

print("Test Accuracy:", test_accuracy)


Test Accuracy: 0.8661971830985915


In [24]:
from sklearn.metrics import accuracy_score

# Train the Random Forest pipeline on the entire training dataset
lr_pipeline.fit(X_train_resampled, y_train_resampled)

# Predict on the test set using the Random Forest pipeline
y_pred_test = lr_pipeline.predict(X_test_tfidf)

# Calculate accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred_test)

print("Test Accuracy:", test_accuracy)


Test Accuracy: 0.8535211267605634


In [25]:
from sklearn.metrics import accuracy_score

# Train the Random Forest pipeline on the entire training dataset
rf_pipeline.fit(X_train_resampled, y_train_resampled)

# Predict on the test set using the Random Forest pipeline
y_pred_test = rf_pipeline.predict(X_test_tfidf)

# Calculate accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred_test)

print("Test Accuracy:", test_accuracy)


Test Accuracy: 0.8788732394366198


In [26]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score

# Define the parameter grid for Random Forest
param_grid = {
    'clf__n_estimators': [100, 200, 300, 400, 500],
    'clf__min_samples_split': [2, 5, 10],
    'clf__min_samples_leaf': [1, 2, 4],
    'clf__criterion': ['gini', 'entropy']  # Example addition
    }

# Create a GridSearchCV object with the Random Forest pipeline and the parameter grid
grid_search = GridSearchCV(rf_pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)

# Fit GridSearchCV to your data
grid_search.fit(X_train_resampled, y_train_resampled)

# Print the best parameters and the best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

# Predict using the best model
y_pred_best = grid_search.predict(X_test_tfidf)  # Make sure to transform your test set features as needed

# Evaluate the best model
print("Accuracy of Best Model:", accuracy_score(y_test, y_pred_best))
print("Classification Report of Best Model:\n", classification_report(y_test, y_pred_best))


Fitting 5 folds for each of 90 candidates, totalling 450 fits
Best Parameters: {'clf__criterion': 'entropy', 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 2, 'clf__n_estimators': 500}
Best Score: 0.984676430908145
Accuracy of Best Model: 0.8830985915492958
Classification Report of Best Model:
               precision    recall  f1-score   support

         0.0       0.80      0.36      0.50       114
         1.0       0.89      0.98      0.93       596

    accuracy                           0.88       710
   macro avg       0.85      0.67      0.72       710
weighted avg       0.88      0.88      0.86       710



In [27]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import classification_report, accuracy_score

# # Define the parameter grid for Logistic Regression
# lr_param_grid = {
#     'clf__C': [0.001, 0.01, 0.1, 1, 10, 100, 200, 300, 400],  # Regularization parameter
#     'clf__penalty': ['l1', 'l2'],  # Regularization penalty
#     'clf__solver': ['liblinear', 'saga'],  # Optimization algorithm
#     'clf__max_iter': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1500, 2000]  # Maximum number of iterations
# }

# # Create a GridSearchCV object with the Logistic Regression pipeline and the parameter grid
# lr_grid_search = GridSearchCV(lr_pipeline, lr_param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)

# # Fit GridSearchCV to your data
# lr_grid_search.fit(X_train_resampled, y_train_resampled)

# # Print the best parameters and the best score
# print("Best Parameters (Logistic Regression):", lr_grid_search.best_params_)
# print("Best Score (Logistic Regression):", lr_grid_search.best_score_)

# # Predict using the best Logistic Regression model
# y_pred_lr_best = lr_grid_search.predict(X_test_tfidf)  # Make sure to transform your test set features as needed

# # Evaluate the best Logistic Regression model
# print("Accuracy of Best Model (Logistic Regression):", accuracy_score(y_test, y_pred_lr_best))
# print("Classification Report of Best Model (Logistic Regression):\n", classification_report(y_test, y_pred_lr_best))


In [28]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score

# Define the parameter grid for Logistic Regression
lr_param_grid = {
'clf__C': [0.001, 0.01, 0.1, 1, 10, 100, 200, 300, 400],  # Regularization parameter
    'clf__penalty': ['l1', 'l2'],  # Regularization penalty
    'clf__solver': ['liblinear', 'saga'],  # Optimization algorithm
    'clf__max_iter': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1500, 2000]  # Maximum number of iterations
}

# Create a GridSearchCV object with the Logistic Regression pipeline and the parameter grid
lr_grid_search = GridSearchCV(lr_pipeline, lr_param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)

# Fit GridSearchCV to your data
lr_grid_search.fit(X_train_resampled, y_train_resampled)

# Print the best parameters and the best score
print("Best Parameters (Logistic Regression):", lr_grid_search.best_params_)
print("Best Score (Logistic Regression):", lr_grid_search.best_score_)

# Predict using the best Logistic Regression model
y_pred_lr_best = lr_grid_search.predict(X_test_tfidf)  # Make sure to transform your test set features as needed

# Evaluate the best Logistic Regression model
print("Accuracy of Best Model (Logistic Regression):", accuracy_score(y_test, y_pred_lr_best))
print("Classification Report of Best Model (Logistic Regression):\n", classification_report(y_test, y_pred_lr_best))


Fitting 5 folds for each of 432 candidates, totalling 2160 fits
Best Parameters (Logistic Regression): {'clf__C': 300, 'clf__max_iter': 1500, 'clf__penalty': 'l1', 'clf__solver': 'saga'}
Best Score (Logistic Regression): 0.9615863659209749
Accuracy of Best Model (Logistic Regression): 0.8873239436619719
Classification Report of Best Model (Logistic Regression):
               precision    recall  f1-score   support

         0.0       0.67      0.58      0.62       114
         1.0       0.92      0.95      0.93       596

    accuracy                           0.89       710
   macro avg       0.80      0.76      0.78       710
weighted avg       0.88      0.89      0.88       710



C:\Users\johns\anaconda3\envs\learn-env\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [29]:
# Define the parameter grid for Multinomial Naive Bayes
nb_param_grid = {
    'clf__alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 1.0],  # Additive smoothing parameter
    'clf__fit_prior': [True, False] # Whether to learn class prior probabilities
}

# Create a GridSearchCV object with the Multinomial Naive Bayes pipeline and the parameter grid
nb_grid_search = GridSearchCV(nb_pipeline, nb_param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)

# Fit GridSearchCV to your data
nb_grid_search.fit(X_train_resampled, y_train_resampled)

# Print the best parameters and the best score
print("Best Parameters (Multinomial Naive Bayes):", nb_grid_search.best_params_)
print("Best Score (Multinomial Naive Bayes):", nb_grid_search.best_score_)

# Predict using the best Multinomial Naive Bayes model
y_pred_nb_best = nb_grid_search.predict(X_test_tfidf)  # Make sure to transform your test set features as needed

# Evaluate the best Multinomial Naive Bayes model
print("Accuracy of Best Model (Multinomial Naive Bayes):", accuracy_score(y_test, y_pred_nb_best))
print("Classification Report of Best Model (Multinomial Naive Bayes):\n", classification_report(y_test, y_pred_nb_best))


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Parameters (Multinomial Naive Bayes): {'clf__alpha': 0.1, 'clf__fit_prior': False}
Best Score (Multinomial Naive Bayes): 0.9378645057183419
Accuracy of Best Model (Multinomial Naive Bayes): 0.8746478873239436
Classification Report of Best Model (Multinomial Naive Bayes):
               precision    recall  f1-score   support

         0.0       0.61      0.62      0.61       114
         1.0       0.93      0.92      0.93       596

    accuracy                           0.87       710
   macro avg       0.77      0.77      0.77       710
weighted avg       0.88      0.87      0.88       710



In [30]:
# Define the parameter grid for Multinomial Naive Bayes
cnb_param_grid = {
    'clf__alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 1.0],  # Additive smoothing parameter
    'clf__fit_prior': [True, False], # Whether to learn class prior probabilities
    'clf__norm': [True, False] # Whether to normalize
}

# Create a GridSearchCV object with the Multinomial Naive Bayes pipeline and the parameter grid
cnb_grid_search = GridSearchCV(cnb_pipeline, cnb_param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)

# Fit GridSearchCV to your data
cnb_grid_search.fit(X_train_resampled, y_train_resampled)

# Print the best parameters and the best score
print("Best Parameters (Multinomial Naive Bayes):", cnb_grid_search.best_params_)
print("Best Score (Multinomial Naive Bayes):", cnb_grid_search.best_score_)

# Predict using the best Multinomial Naive Bayes model
y_pred_nb_best = cnb_grid_search.predict(X_test_tfidf)  # Make sure to transform your test set features as needed

# Evaluate the best Multinomial Naive Bayes model
print("Accuracy of Best Model (Multinomial Naive Bayes):", accuracy_score(y_test, y_pred_nb_best))
print("Classification Report of Best Model (Multinomial Naive Bayes):\n", classification_report(y_test, y_pred_nb_best))


Fitting 5 folds for each of 24 candidates, totalling 120 fits
Best Parameters (Multinomial Naive Bayes): {'clf__alpha': 0.1, 'clf__fit_prior': True, 'clf__norm': False}
Best Score (Multinomial Naive Bayes): 0.9378645057183419
Accuracy of Best Model (Multinomial Naive Bayes): 0.8746478873239436
Classification Report of Best Model (Multinomial Naive Bayes):
               precision    recall  f1-score   support

         0.0       0.61      0.62      0.61       114
         1.0       0.93      0.92      0.93       596

    accuracy                           0.87       710
   macro avg       0.77      0.77      0.77       710
weighted avg       0.88      0.87      0.88       710



In [31]:
# # Create a dictionary of emojis
# emojis= {
# '😒' : '[grumpy]',
# '😡' : '[angry]',
# '🤬' : '[cursing]',
# '😞' : '[disappointed]',
# '😤' : '[frustrated]',
# '😭' : '[crying]',
# '🤦‍♂️' : '[facepalmboy]',
# '😑' : '[displeased]',
# '😢' : '[tearful]',
# '😱' : '[spooked]',
# '😂' : '[laughing]',
# '😄' : '[smiling]',
# '😝' : '[silly]',
# '🙄' : '[eyeroll]',
# '😏' : '[smirking]',
# '🤔' : '[confused]',
# '🗳️' : '[voting]',
# '📈' : '[stockup]',
# '📉' : '[stockdown]',
# '😩' : '[distressed]',
# '😠' : '[mad]',
# '😓' : '[unhappy]',
# '🤣' : '[laughter]',
# '😕' : '[uhoh]',
# '🙅‍♂️' : '[crossarmsblue]',
# '🤳' : '[arm]',
# '🙅‍♀️' : '[crossarmspurple]',
# '🔋' : '[battery]',
# '😆' : '[bigsmile]',
# '🤦‍♀️' : '[facepalmgirl]',
# '📱' : '[phone]',
# '😫' : '[distressed]',
# '😅' : '[lmao]',
# '🤨' : '[raisedeyebrow]',
# '😔' : '[sad]',
# '😖' : '[yikes]'
# }

# # Create a function to convert emojis to text
# def emoji_fixer(x):
#     for k in emojis.keys():
#         x=x.replace(k, emojis[k])
#     return x

# # Apply the function to tweets column
# neg_tweets['tweet']=neg_tweets['tweet'].apply(emoji_fixer)